In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import os

In [16]:
gdp_df = pd.read_csv(r"/content/drive/MyDrive/CS_Finance_Data/MacroEconomic_Variables/GDP.csv", parse_dates=True)
gdp_df['DATE'] = pd.to_datetime(gdp_df['DATE'])
print(gdp_df.shape)
gdp_df.head()

(81, 2)


,DATE,GDP
0,2004-01-01,11923.447
1,2004-04-01,12112.815
2,2004-07-01,12305.307
3,2004-10-01,12527.214
4,2005-01-01,12767.286


In [17]:
uner_df = pd.read_csv(r"/content/drive/MyDrive/CS_Finance_Data/MacroEconomic_Variables/UNRATE.csv")
uner_df['DATE'] = pd.to_datetime(uner_df['DATE'])
print(uner_df.shape)
uner_df.head()

(82, 2)


,DATE,UNRATE
0,2004-01-01,5.700000
1,2004-04-01,5.600000
2,2004-07-01,5.433333
3,2004-10-01,5.433333
4,2005-01-01,5.300000


In [18]:
df_gdp_uner = pd.merge(gdp_df,uner_df,on='DATE')
print(df_gdp_uner.shape)
df_gdp_uner.head()

(81, 3)


,DATE,GDP,UNRATE
0,2004-01-01,11923.447,5.700000
1,2004-04-01,12112.815,5.600000
2,2004-07-01,12305.307,5.433333
3,2004-10-01,12527.214,5.433333
4,2005-01-01,12767.286,5.300000


In [19]:
df_gdp_uner.rename(columns={'DATE': 'date'}, inplace=True)

In [20]:
def update_quart_enddate_shift(df):
    # Define a dictionary to map the month to the new date
    quarter_end_dates = {1: (12, 31), 4: (3, 31), 7: (6, 30), 10: (9, 30)}

    # Apply the mapping to update the dates
    for month, (new_month, new_day) in quarter_end_dates.items():
        df.loc[df['date'].dt.month == month, 'date'] = df['date'].apply(
            lambda x: x.replace(month=new_month, day=new_day) if x.month == month else x
        )

    # offset the years by 1 for those whose month is december
    tmp_ids = df[df["date"].dt.month == 12].index
    df.loc[tmp_ids, "date"] = df.loc[tmp_ids, "date"] - pd.DateOffset(years=1)

    # drop the first row df_gdp_uner
    df.drop(df.index[0], inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [21]:
df_gdp_uner =  update_quart_enddate_shift(df_gdp_uner)
df_gdp_uner.head()

,date,GDP,UNRATE
0,2004-03-31,12112.815,5.600000
1,2004-06-30,12305.307,5.433333
2,2004-09-30,12527.214,5.433333
3,2004-12-31,12767.286,5.300000
4,2005-03-31,12922.656,5.100000


In [22]:
# add a year column to df_gdp_uner
df_gdp_uner['Year'] = df_gdp_uner['date'].dt.year
df_gdp_uner.head()

,date,GDP,UNRATE,Year
0,2004-03-31,12112.815,5.600000,2004
1,2004-06-30,12305.307,5.433333,2004
2,2004-09-30,12527.214,5.433333,2004
3,2004-12-31,12767.286,5.300000,2004
4,2005-03-31,12922.656,5.100000,2005


In [23]:
# Assign quarter
def assign_quarter(date):
    month = date.month
    if month == 3:
        return 'Q1'
    elif month == 6:
        return 'Q2'
    elif month == 9:
        return 'Q3'
    else:
        return 'Q4'


In [24]:
df_gdp_uner['Quarter'] = df_gdp_uner['date'].apply(assign_quarter)
df_gdp_uner.head()

,date,GDP,UNRATE,Year,Quarter
0,2004-03-31,12112.815,5.600000,2004,Q1
1,2004-06-30,12305.307,5.433333,2004,Q2
2,2004-09-30,12527.214,5.433333,2004,Q3
3,2004-12-31,12767.286,5.300000,2004,Q4
4,2005-03-31,12922.656,5.100000,2005,Q1
